# Reinforcement Learning

Im Gegensatz zum konventionellen ML wird beim RL kein vorgegebenes Datenset verwendet

Beim RL gibt es einen Prozess der erlernt werden soll, während des Prozesses werden die Daten erzeugt

Prozess:
- Was soll gelernt werden?
    - Reale Prozesse (Roboter)
    - Softwareprozesse (Spiele)
 
Umgebung:
- Der Prozess selbst
- Enthält Funktionen, welche der Agent verwenden kann, um den Prozess auszuführen

Agent:
- "Der Akteur", welcher den Prozess ausführt
- Lernt aus den Daten, welche vom Prozess erzeugt werden

Pakete:
- Gym(nasium)
- Stable Baselines 3

In [1]:
import gymnasium as gym

## Vorgegebene Umgebung

Innerhalb von Gym gibt es verschiedene Testumgebungen, um RL zu probieren

Eine davon heißt CartPole-V1

Es geht darum ein Kart zu bewegen, um eine darauf angebrachte Stange gerade zu halten

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [10]:
env = gym.make("CartPole-v1", render_mode="human")

### Inhalte einer Umgebung

Daten:
- Action Space: Gibt dem Agenten vor, was er tun kann
| Num | Action                 |
|-----|------------------------|
| 0   | Push cart to the left  |
| 1   | Push cart to the right |
 
- Observation Space: Gibt die Daten vor, welche für das ML verwendet können

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

---

Funktionen der Umgebung:
- step: Bringt den Prozess um einen Schritt weiter
- reset: Setzt die Umgebung auf den Anfangszustand zurück
- render: Zeichnet die Umgebung (optional)
- close: Wird ausgeführt, wenn die Umgebung abgebaut wird (optional)

In [11]:
durchgaenge = 5
for d in range(durchgaenge):
    start = env.reset()  # Am Anfang von jedem Durchgang wird die Umgebung zurückgesetzt
    score = 0  # Summiert die Belohnung nach jedem Step auf
    done = False

    while not done:  # Führe die Umgebung aus, bis die step Methode True zurück gibt
        env.render()  # Umgebung zeichnen
        action = env.action_space.sample()  # Wähle eine zufällige Action, und füge diese bei step ein
        newState, reward, done, term, info = env.step(action)  # Führe die Umgebung einmal aus
        score += reward
    print(f"Durchgang: {d}, Score: {score}")
env.close()  # Fenster schließen

Durchgang: 0, Score: 86.0
Durchgang: 1, Score: 12.0
Durchgang: 2, Score: 33.0
Durchgang: 3, Score: 19.0
Durchgang: 4, Score: 18.0


Dieser Agent wählt zufällige Actions

Hier soll jetzt ein ML Modell dahinterstehen, welches sinnvolle Actions auswählt

## RL Modell trainieren

Stable Baselines 3 stellt uns einige vordefinierte Algorithmen bereit

Einer davon ist PPO (Proximal Policy Optimization)

Wir benötigen zusätzlich noch eine Policy (definiert, wie das unterliegende Tensorflow Modell aussieht)

Zusatz: DummyVecEnv, ermöglicht Parallelisierung des Lernprozesses

In [12]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [22]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device


In [23]:
model.learn(total_timesteps=50_000)

-----------------------------
| time/              |      |
|    fps             | 983  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 480         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008324099 |
|    clip_fraction        | 0.0785      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00605    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.04        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0098     |
|    value_loss           | 46.3        |
-----------------------------------------
----------------------------------

In [24]:
model.save("CartPole")

In [38]:
env = gym.make("CartPole-v1", render_mode="human")

In [39]:
model = PPO.load("CartPole", env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [41]:
durchgaenge = 5
for d in range(durchgaenge):
    state = env.reset()  # Am Anfang von jedem Durchgang wird die Umgebung zurückgesetzt
    score = 0  # Summiert die Belohnung nach jedem Step auf
    done = False
    
    while not done:  # Führe die Umgebung aus, bis die step Methode True zurück gibt
        env.render()
        if len(state) == 2:
            action, _ = model.predict(state[0])
        else:
            action, _ = model.predict(state)
        state, reward, done, term, info = env.step(action)  # Führe die Umgebung einmal aus
        score += reward

        if score > 100:
            break
    print(f"Durchgang: {d}, Score: {score}")
env.close()  # Fenster schließen

Durchgang: 0, Score: 101.0
Durchgang: 1, Score: 101.0
Durchgang: 2, Score: 101.0
Durchgang: 3, Score: 101.0
Durchgang: 4, Score: 101.0
